In [170]:
with open("day20.txt", "r") as f:
    lines = [line.strip() for line in f.readlines()]

In [171]:
lines

['%hf -> qk',
 '%xp -> pn, mq',
 '&rz -> lb',
 '%nm -> hn',
 '%zb -> gz',
 '&lf -> lb',
 '%nn -> vd, zc',
 '%xn -> fz, kb',
 '%gz -> vf',
 '&pn -> lz, hq, lf, mh, bh, mq',
 '%xb -> th',
 '%vf -> nc, vd',
 '%ds -> kz, pn',
 '&br -> lb',
 '%cm -> fm',
 '%qz -> bh, pn',
 '&lb -> rx',
 '%vx -> pn, ds',
 '%kz -> pn',
 '%gp -> cm, th',
 '%hq -> mh',
 '%fq -> xd',
 '%mj -> th, np',
 '%lz -> vq, pn',
 '%hn -> xn, fz',
 '%fl -> fz, rq',
 '%fm -> hh, th',
 '%tx -> fz, qn',
 '%mh -> xp',
 '%dn -> nm, fz',
 '%xv -> vd',
 '&vd -> zc, nn, hf, br, zb, tp, gz',
 '%np -> fq',
 '%sf -> vd, xv',
 '%rq -> fn, fz',
 '%zc -> ms',
 '&fk -> lb',
 '%qn -> fz, bt',
 '%qk -> vd, zb',
 '%ms -> tp, vd',
 '%xd -> th, gp',
 '%hh -> th, dq',
 '%sx -> th, xb',
 '%fn -> fz',
 '%jd -> pn, vx',
 '%mq -> jd',
 '&th -> mj, rz, np, fq, cm',
 '%bt -> fz, dn',
 '%dq -> dj, th',
 '%tp -> hf',
 '%nc -> sf, vd',
 'broadcaster -> nn, lz, mj, tx',
 '%bh -> hq',
 '&fz -> fk, nm, tx',
 '%cr -> fl, fz',
 '%vq -> qz, pn',
 '%dj -> th,

In [172]:
def parse_lines(lines):
    node_map = {}
    for line in lines:
        source, targets = line.split(" -> ")
        tmp_map = {}
        state = "none"
        # get source & source type
        source_type = "none"
        if source == "broadcaster":
            source_type = "broadcaster"
        elif source.startswith("%"):
            source_type = "flipflop"
            source = source[1:]
            state = "off"
            tmp_map["state"] = state
        elif source.startswith("&"):
            source_type = "conjunction"
            source = source[1:]
            memory = {}
            tmp_map["memory"] = memory
        # build targets list
        targets = targets.split(", ")
        # add to node map
        node_map[source] = {}
        node_map[source]["type"] = source_type
        node_map[source]["targets"] = targets
        node_map[source]["from"] = [] 
        node_map[source]["name"] = source
        for k,v in tmp_map.items():
            node_map[source][k] = v

    # check if all targets exist
    new_node_map = {}
    for node, node_info in node_map.items():
        for target in node_info["targets"]:
            if target not in node_map:
                new_node_map[target] = {}
                new_node_map[target]["type"] = "none"
                new_node_map[target]["targets"] = []
                new_node_map[target]["from"] = []
                new_node_map[target]["name"] = target
                print(f"Node {target} not found, creating...")
    node_map.update(new_node_map)

    # also build a from list
    for node, node_info in node_map.items():
        for target in node_info["targets"]:
            if target not in node_map:
                raise Exception("Target not found")
            node_map[target]["from"].append(node)

    # update initial memory for conjunctions
    for node, node_info in node_map.items():
        if node_info["type"] == "conjunction":
            for from_node in node_info["from"]:
                node_info["memory"][from_node] = "off"
    return node_map
        

In [173]:
node_map = parse_lines(lines)
node_map

Node rx not found, creating...


{'hf': {'type': 'flipflop',
  'targets': ['qk'],
  'from': ['vd', 'tp'],
  'name': 'hf',
  'state': 'off'},
 'xp': {'type': 'flipflop',
  'targets': ['pn', 'mq'],
  'from': ['mh'],
  'name': 'xp',
  'state': 'off'},
 'rz': {'type': 'conjunction',
  'targets': ['lb'],
  'from': ['th'],
  'name': 'rz',
  'memory': {'th': 'off'}},
 'nm': {'type': 'flipflop',
  'targets': ['hn'],
  'from': ['dn', 'fz'],
  'name': 'nm',
  'state': 'off'},
 'zb': {'type': 'flipflop',
  'targets': ['gz'],
  'from': ['vd', 'qk'],
  'name': 'zb',
  'state': 'off'},
 'lf': {'type': 'conjunction',
  'targets': ['lb'],
  'from': ['pn'],
  'name': 'lf',
  'memory': {'pn': 'off'}},
 'nn': {'type': 'flipflop',
  'targets': ['vd', 'zc'],
  'from': ['vd', 'broadcaster'],
  'name': 'nn',
  'state': 'off'},
 'xn': {'type': 'flipflop',
  'targets': ['fz', 'kb'],
  'from': ['hn'],
  'name': 'xn',
  'state': 'off'},
 'gz': {'type': 'flipflop',
  'targets': ['vf'],
  'from': ['zb', 'vd'],
  'name': 'gz',
  'state': 'off'},
 

In [174]:
def process_broadcast(node, from_node, pulse):
    events = []
    for target in node["targets"]:
        # event: source, target, pulse
        # just continue redirect
        events.append((node["name"], target, pulse))
    return events

In [175]:
def process_flipflop(node, from_node, pulse):
    events = []

    if pulse == "high":
        # nothing happens
        return events
    elif pulse == "low":
        # flip state
        if node["state"] == "off":
            node["state"] = "on"
        elif node["state"] == "on":
            node["state"] = "off"
        else:
            raise Exception(f"Invalid state {node['state']}")


        # send out
        out_pulse = "high" if node["state"] == "on" else "low"
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, out_pulse))
        return events

In [176]:
def process_none(node, from_node, pulse):
    return []

In [177]:
def process_conjunction(node, from_node, pulse):
    # first update memory for that input
    node["memory"][from_node] = pulse

    events = []
    # if high on all memory, send out low
    if all([v == "high" for v in node["memory"].values()]):
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, "low"))
        return events
    else:
        # send a high to all targets
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, "high"))
        return events

In [178]:
# queue = []
# queue.append(("button", "broadcaster", "low"))

# iter_step = 0
# while len(queue) > 0 and iter_step < 20:
#     iter_step += 1
#     source, target, pulse = queue.pop(0)
#     print(f"{source} --({pulse})--> {target}")
#     node = node_map[target]
#     # print("node: ", node)
#     if node["type"] == "broadcaster":
#         queue.extend(process_broadcast(node, source, pulse))
#     elif node["type"] == "flipflop":
#         queue.extend(process_flipflop(node, source, pulse))
#     elif node["type"] == "none":
#         queue.extend(process_none(node, source, pulse))
#     elif node["type"] == "conjunction":
#         queue.extend(process_conjunction(node, source, pulse))
#     else:
#         raise Exception(f"Invalid node type {node['type']}")
#     # print(f"Queue: {queue}")

In [179]:
total_high_pulse_count = 0
total_low_pulse_count = 0

for run_idx in range(1000):
    if run_idx % 100 == 0:
        print(f"Run {run_idx}")
    queue = []
    queue.append(("button", "broadcaster", "low"))

    iter_step = 0
    while len(queue) > 0 and iter_step < 100000:
        if iter_step > 80000:
            print(f"Run {run_idx}, iter {iter_step}")
        iter_step += 1
        source, target, pulse = queue.pop(0)
        # print(f"{source} --({pulse})--> {target}")

        if pulse == "high":
            total_high_pulse_count += 1
        elif pulse == "low":
            total_low_pulse_count += 1

        node = node_map[target]
        # print("node: ", node)
        if node["type"] == "broadcaster":
            queue.extend(process_broadcast(node, source, pulse))
        elif node["type"] == "flipflop":
            queue.extend(process_flipflop(node, source, pulse))
        elif node["type"] == "none":
            queue.extend(process_none(node, source, pulse))
        elif node["type"] == "conjunction":
            queue.extend(process_conjunction(node, source, pulse))
        else:
            raise Exception(f"Invalid node type {node['type']}")
            # print(f"Queue: {queue}")

total_high_pulse_count * total_low_pulse_count

Run 0
Run 100
Run 200
Run 300
Run 400
Run 500
Run 600
Run 700
Run 800
Run 900


817896682